<a href="https://colab.research.google.com/github/KingAbdulRehman/LangChain/blob/main/LangChain_ToolCalling/LangChain_Calculator_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction:

Setup Gemeni Api key as **"GOOGLE_API_KEY"** in Google Colab Secret


# Install and Import Packages

In [ ]:
!pip install langchain-google-genai langchain

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata
from langchain_core.tools import tool, StructuredTool
from pydantic import BaseModel, Field
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage, FunctionMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from rich.console import Console
from rich.markdown import Markdown
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Setup API Keys

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Setup LLM

In [4]:
llm = ChatGoogleGenerativeAI(
  model="gemini-1.5-pro",
  max_tokens=1000,
  max_retries=3,
  top_k=2,
  temperature=0.8
)

In [5]:
# test llm
print(llm.invoke("Hello"))

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-8bd30c0c-5a20-400e-a13e-405f7bb90f70-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


# Create Calculator Tool

In [6]:
class CalculatorModel(BaseModel):
    expression: str = Field(description="math expression")


class Calculator:
    def calculate(self, expression: str):
        try:
            result = eval(expression, {"__builtins__": None}, {})
            return result
        except Exception as e:
            return f"Error: {e}"



def calculator(expression: str):
    """A mathematical expression as a string (e.g., "2 + 2")."""
    calc = Calculator()
    result = calc.calculate(expression)
    return ("Calculator-Tool", result)


calculator_tool = StructuredTool.from_function(
    func=calculator,
    name="Calculator-Tool",
    description="A mathematical expression as a string (e.g., \"2 + 2\").",
    args_schema= CalculatorModel
)

In [7]:
# test tool
print(calculator_tool.name)
print(calculator_tool.description)
print(calculator_tool.args)
print(calculator_tool.args_schema.model_json_schema())
calculator_tool.invoke("(2 + 2)-(6 + 4)")

Calculator-Tool
A mathematical expression as a string (e.g., "2 + 2").
{'expression': {'description': 'math expression', 'title': 'Expression', 'type': 'string'}}
{'properties': {'expression': {'description': 'math expression', 'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'title': 'CalculatorModel', 'type': 'object'}


('Calculator-Tool', -6)

# Create Tool Calling Agent

In [9]:
from typing import Any, Dict, List, Optional, Union
tools = [calculator_tool]

prompt = hub.pull("hwchase17/openai-tools-agent")

def _format_function_message(
    name: str,
    result: Any,
) -> FunctionMessage:
    """Format message to look like a function call."""
    return FunctionMessage(
        name=name,
        content=str(result),
    )

agent = create_tool_calling_agent(
  llm=llm,
  tools=tools,
  prompt=prompt,
  message_formatter=_format_function_message
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
ai_search_res = agent_executor.invoke({"input": "Please solve: 2 * 5 + 7 / 18 and please explain"})



> Entering new AgentExecutor chain...


TypeError: _format_function_message() missing 1 required positional argument: 'result'

In [ ]:
console = Console()
console.print(Markdown(str(ai_search_res["output"])))

# Add Chat history

In [108]:
memory = InMemoryChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output"  # Ensure this key matches the output of your agent
)

config = {"configurable": {"session_id": "test-session"}}

In [109]:
print(
    agent_with_chat_history.invoke(
        {"input": "Please solve 10 * 20 * 100 / 150"}, config
    )["output"]
)



> Entering new AgentExecutor chain...



Invoking: `Calculator-Tool` with `{'expression': '10 * 20 * 100 / 150'}`


133.33333333333334


> Finished chain.
133.33333333333334


In [110]:
print(
    agent_with_chat_history.invoke(
        {"input": "did you tell me what i ask previosuly?"}, config
    )["output"]
)



> Entering new AgentExecutor chain...
I'm sorry, I don't have memory of past conversations. If you'd like me to answer your previous question, could you please repeat it?

> Finished chain.
I'm sorry, I don't have memory of past conversations. If you'd like me to answer your previous question, could you please repeat it?
